# aging simulaiton of modules with temperature data

This notebook explores the aging of modules with different thermal behaviour in the FCR application.  
The results are stored and evaluated in the Notebook Evaluaions

In [90]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import math
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from simses.main import SimSES
from configparser import ConfigParser

In [91]:
pd.options.plotting.backend = "plotly"
# template = "plotly_dark"
template = "plotly_white"

## File directory

In [92]:
os.path.abspath("./data")

'c:\\Users\\granermm\\OneDrive - Hochschule für angewandte Wissenschaften Kempten\\General\\03 Vivek\\IRES\\JoES IRES Data\\data'

## setpoint profile


In [93]:
profile_setpoints = pd.read_csv("./Power setpoints all strategies/output_MPC_ires.csv",header=0)
profile_setpoints.head()


,power_tu1,power_charge1,power_discharge1,power_loss1,soe1,power_tu2,power_charge2,power_discharge2,power_loss2,soe2
0,6.000000,0.000,6.00000,0.00000,0.500000,4.015847,0.000000,4.015847,0.000000,0.200000
1,0.000000,0.000,0.00000,0.00000,0.499500,5.861722,5.861722,0.000000,0.000000,0.199665
2,6.000000,1.374,0.00000,4.62600,0.499500,30.454265,25.314725,0.000000,5.139540,0.200154
3,6.000000,1.374,0.00000,4.62600,0.499615,39.924146,34.585739,0.000000,5.338407,0.202263
4,164.183311,0.000,172.13116,7.94785,0.499729,6.000000,0.000000,10.626000,4.626000,0.205146


In [94]:
num_rows = len(profile_setpoints)  
profile_setpoints['time'] = pd.date_range(start='2014-01-01 00:00:00', periods=num_rows, freq='15T')
#profile_setpoints.set_index('time', inplace=True)
profile_setpoints.head()

,power_tu1,power_charge1,power_discharge1,power_loss1,soe1,power_tu2,power_charge2,power_discharge2,power_loss2,soe2,time
0,6.000000,0.000,6.00000,0.00000,0.500000,4.015847,0.000000,4.015847,0.000000,0.200000,2014-01-01 00:00:00
1,0.000000,0.000,0.00000,0.00000,0.499500,5.861722,5.861722,0.000000,0.000000,0.199665,2014-01-01 00:15:00
2,6.000000,1.374,0.00000,4.62600,0.499500,30.454265,25.314725,0.000000,5.139540,0.200154,2014-01-01 00:30:00
3,6.000000,1.374,0.00000,4.62600,0.499615,39.924146,34.585739,0.000000,5.338407,0.202263,2014-01-01 00:45:00
4,164.183311,0.000,172.13116,7.94785,0.499729,6.000000,0.000000,10.626000,4.626000,0.205146,2014-01-01 01:00:00


In [95]:
profile_setpoints['power1'] = profile_setpoints['power_charge1'] - profile_setpoints['power_discharge1']
profile_setpoints['power2'] = profile_setpoints['power_charge2'] - profile_setpoints['power_discharge2']
profile_setpoints

,power_tu1,power_charge1,power_discharge1,power_loss1,soe1,power_tu2,power_charge2,power_discharge2,power_loss2,soe2,time,power1,power2
0,6.000000,0.000000,6.000000,0.000000,0.500000,4.015847,0.000000,4.015847,0.000000,0.200000,2014-01-01 00:00:00,-6.000000,-4.015847
1,0.000000,0.000000,0.000000,0.000000,0.499500,5.861722,5.861722,0.000000,0.000000,0.199665,2014-01-01 00:15:00,0.000000,5.861722
2,6.000000,1.374000,0.000000,4.626000,0.499500,30.454265,25.314725,0.000000,5.139540,0.200154,2014-01-01 00:30:00,1.374000,25.314725
3,6.000000,1.374000,0.000000,4.626000,0.499615,39.924146,34.585739,0.000000,5.338407,0.202263,2014-01-01 00:45:00,1.374000,34.585739
4,164.183311,0.000000,172.131160,7.947850,0.499729,6.000000,0.000000,10.626000,4.626000,0.205146,2014-01-01 01:00:00,-172.131160,-10.626000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2299,66.766927,60.864821,0.000000,5.902105,0.193661,66.766927,60.864821,0.000000,5.902105,0.193661,2014-01-24 22:45:00,60.864821,60.864821
2300,81.262402,0.000000,87.468913,6.206510,0.198733,81.262402,0.000000,87.468913,6.206510,0.198733,2014-01-24 23:00:00,-87.468913,-87.468913
2301,3.652665,0.000000,3.652665,0.000000,0.191444,3.652665,0.000000,3.652665,0.000000,0.191444,2014-01-24 23:15:00,-3.652665,-3.652665
2302,22.053069,0.000000,27.016184,4.963114,0.191140,22.053069,0.000000,27.016184,4.963114,0.191140,2014-01-24 23:30:00,-27.016184,-27.016184


In [7]:
#adjust building of power column as charge/discharge power were not available, if simses shall be used

def process_and_save_profiles(input_folder, output_folder):
    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)
    
    # Get list of all CSV files in the input directory
    csv_files = [f for f in os.listdir(input_folder) if f.endswith('.csv')]
    
    for file_name in csv_files:
        file_path = os.path.join(input_folder, file_name)
        
        # Load the CSV file
        df = pd.read_csv(file_path, header=0)
        
        # Add time column
        num_rows = len(df)
        df['time'] = pd.date_range(start='2014-01-01 00:00:00', periods=num_rows, freq='15T')
        df['power1'] = df['power_tu1']
        # Iterate over the possible numeric suffixes
        
        if f'power1' in df.columns:
            # Select the relevant column
            df_subset = df[['time', f'power1']]
            
            # Construct the output file name
            output_file_name = f'{file_name[:-4]}_power.csv'  # Excludes ".csv" from file_name
            output_file_path = os.path.join(output_folder, output_file_name)
            
            # Save the subset to a CSV file
            df_subset.to_csv(output_file_path, index=False, header=False)
            print(f'Saved {output_file_name} to {output_file_path}')
        else:
            print(f'Column power not found in {file_name}')

# Example usage:
input_folder = os.path.abspath("./Power setpoints all strategies")
output_folder = os.path.abspath("./data/strategy_setpoints_module4")

process_and_save_profiles(input_folder, output_folder)


Saved output_ConsqDist_power.csv to c:\Users\granermm\OneDrive - Hochschule für angewandte Wissenschaften Kempten\General\03 Vivek\IRES\JoES IRES Data\data\strategy_setpoints_module4\output_ConsqDist_power.csv
Saved output_EqualDist_power.csv to c:\Users\granermm\OneDrive - Hochschule für angewandte Wissenschaften Kempten\General\03 Vivek\IRES\JoES IRES Data\data\strategy_setpoints_module4\output_EqualDist_power.csv
Saved output_MPC_ires_power.csv to c:\Users\granermm\OneDrive - Hochschule für angewandte Wissenschaften Kempten\General\03 Vivek\IRES\JoES IRES Data\data\strategy_setpoints_module4\output_MPC_ires_power.csv
Saved output_MPC_ires_120days_power.csv to c:\Users\granermm\OneDrive - Hochschule für angewandte Wissenschaften Kempten\General\03 Vivek\IRES\JoES IRES Data\data\strategy_setpoints_module4\output_MPC_ires_120days_power.csv
Saved output_ruleBased_power.csv to c:\Users\granermm\OneDrive - Hochschule für angewandte Wissenschaften Kempten\General\03 Vivek\IRES\JoES IRES Da

temperature in 30s resolution --> mean to 15 min resolution

In [288]:
module4_temperatures = pd.read_csv("./T evolution all strategies/00. M4_temp_allstrategies.csv",header=0)
module4_temperatures.head()
module_temperatures4=module4_temperatures.drop(columns=['Time'])
module_temperatures4_15min=module_temperatures4.groupby(module_temperatures4.index // 30).mean()
module_temperatures4_15min.head()

,Temperature_M4_MPC,Temperature_M4_Equal,Temperature_M4_Conseq,Temperature_M4_RuleBased
0,295.000035,295.000041,295.000162,295.000069
1,295.000073,295.000098,295.000391,295.000148
2,295.000074,295.000650,295.002599,295.000419
3,295.000078,295.002075,295.008301,295.001119
4,295.029069,295.014693,295.058771,295.021281


save subsets of modules to data folder
we need the time and power for the simulation

save subsets of module temperature arrays in separate files

In [45]:
def process_and_save_temp_data(input_folder, output_folder):
    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)
    
    # Get list of all CSV files in the input directory
    csv_files = [f for f in os.listdir(input_folder) if f.endswith('.csv')]
    
    for file_name in csv_files:
        file_path = os.path.join(input_folder, file_name)
        
        # Load the CSV file
        df = pd.read_csv(file_path, header=0)
        df=df.drop(columns=['Time'])
        df=df.groupby(df.index // 30).mean()
        # Iterate over all columns in the DataFrame
        for column_name in df.columns:
            # Check if the column name contains 'Temperature_MPC_M'
            if 'Temperature_M' in column_name:
                # Select the current column
                df_subset = df[[column_name]].copy()
                
                # Convert from Kelvin to Celsius
                df_subset[column_name] = df_subset[column_name] - 273.15
                
                # Construct the output file name by including both the original file name and column name
                # Strip ".csv" from the original file name
                base_file_name = os.path.splitext(file_name)[0]
                output_file_name = f'{base_file_name}_{column_name}.csv'
                output_file_path = os.path.join(output_folder, output_file_name)
                
                # Save the subset to a CSV file
                df_subset.to_csv(output_file_path, index=False, header=False)
                print(f'Saved {output_file_name} to {output_file_path}')
            else:
                print(f'Column {column_name} does not match the pattern in {file_name}')

# Example usage:
input_folder = os.path.abspath("./T evolution all strategies")
output_folder = os.path.abspath("./data/strategy_temperature_module4")

process_and_save_temp_data(input_folder, output_folder)

Saved 00. M4_temp_allstrategies_Temperature_M4_MPC.csv to c:\Users\granermm\OneDrive - Hochschule für angewandte Wissenschaften Kempten\General\03 Vivek\IRES\JoES IRES Data\data\strategy_temperature_module4\00. M4_temp_allstrategies_Temperature_M4_MPC.csv
Saved 00. M4_temp_allstrategies_Temperature_M4_Equal.csv to c:\Users\granermm\OneDrive - Hochschule für angewandte Wissenschaften Kempten\General\03 Vivek\IRES\JoES IRES Data\data\strategy_temperature_module4\00. M4_temp_allstrategies_Temperature_M4_Equal.csv
Saved 00. M4_temp_allstrategies_Temperature_M4_Conseq.csv to c:\Users\granermm\OneDrive - Hochschule für angewandte Wissenschaften Kempten\General\03 Vivek\IRES\JoES IRES Data\data\strategy_temperature_module4\00. M4_temp_allstrategies_Temperature_M4_Conseq.csv
Saved 00. M4_temp_allstrategies_Temperature_M4_RuleBased.csv to c:\Users\granermm\OneDrive - Hochschule für angewandte Wissenschaften Kempten\General\03 Vivek\IRES\JoES IRES Data\data\strategy_temperature_module4\00. M4_te

# General simulation configuration

NSE-code

In [156]:
#load data from original df, as it includes soe instead of power and the header
profile_setpoints_ConseqDist = pd.read_csv("./Power setpoints all strategies/output_ConsqDist.csv",header=0)
profile_setpoints_EqualDist = pd.read_csv("./Power setpoints all strategies/output_EqualDist.csv",header=0)
profile_setpoints_ruleBased = pd.read_csv("./Power setpoints all strategies/output_ruleBased.csv",header=0)

In [157]:
string_profiles = {
    "MPC": profile_setpoints, #solange power_tu verwendet wird für erate
    "Conseq" : profile_setpoints_ConseqDist,
    "Equal": profile_setpoints_EqualDist, 
    "rulebased": profile_setpoints_ruleBased
    
}

In [158]:
# depth of discharge cycle count
def calc_cycle_depth(df):
    soc = df["soe1"].to_numpy() 
    soc_diff = np.diff(soc) 
    n = len(soc_diff)

    soc_sign_diff = np.diff(np.sign(soc_diff)) # identify where the sign changes in the "soc_diff" array.
    soc_sign_diff = np.append(0, soc_sign_diff) # align with soc_diff # Prepends a 0 to the "soc_sign_diff" array to align it with the "soc_diff" array, ensuring that both arrays have the same length.
    nonzeros = np.nonzero(soc_sign_diff)[0] # find indices of non-zero elements in the "s_sign_diff" array. These indices correspond to positions where the sign of the differences in "soc" changes.
    start = np.append(0, nonzeros) #Prepends a 0 to the "nonzeros" array and assigns the result to "start." This is used to mark the start positions of each segment where the sign of the differences changes.
    stop  = np.append(nonzeros, n) #Appends the last index of "s_diff" (which is "n") to the "nonzeros" array and assigns the result to "stop." This is used to mark the end positions of each segment where the sign of the differences changes.

    x = len(start) #The variable x is set to the number of elements in the start array, which is essentially the number of detected cycles in the data
    soc_change = np.zeros(x) #An array named s_change is initialized with zeros, and it will be used to store the accumulated changes within each cycle.
    for i in range(x):
        soc_change[i] = np.sum(soc_diff[start[i]:stop[i]])
        #This loop iterates through each detected cycle and calculates the total change (s_change[i]) within that cycle /
        # by summing the corresponding elements in the s_diff array.

    start = start[soc_change > 0]
    soc_change = soc_change[soc_change > 0]
    #After the loop, the arrays start and s_change are filtered to include only cycles with positive changes (non-zero). This step helps exclude cycles where there is no meaningful change in the data.

    idx = df.index[start] # It retrieves the indices in the original DataFrame df corresponding to the filtered start indices.
    df.loc[idx, "doc1"] = soc_change #The doc cycle count values (s_change) for the selected cycles are stored in a new column named "doc" in the DataFrame df.

    df.fillna(0.0, inplace=True)#Any missing values (NaNs) in the DataFrame are filled with zeros.

    return #The function does not have a return statement, so it implicitly returns None.

In [217]:
capacity=3000/7 #kWh
#power in profiles is in kW

In [218]:
#change back to power1
def calc_erate(df):
    df["erate1"] = df["power_tu1"].abs() / capacity # absoulte Value of power

In [219]:
#verm quatsch, da power_tu nur positiv
def calc_erate_mpc(df):
    df["erate1"] = df["power1"].abs() / capacity # absoulte Value of power
calc_erate_mpc(profile_setpoints)
calc_cycle_depth(profile_setpoints)

In [220]:
calc_erate(profile_setpoints_ConseqDist)
calc_cycle_depth(profile_setpoints_ConseqDist)
#profile_setpoints.head()

In [221]:
calc_erate(profile_setpoints_EqualDist)
calc_cycle_depth(profile_setpoints_EqualDist)

In [222]:
calc_erate(profile_setpoints_ruleBased)
calc_cycle_depth(profile_setpoints_ruleBased)

In [223]:
def lfp_calendar(df, years=1):
    """Calendar-degradation model of LFP cell.
    
    Parameters
    ----------
    df: pandas-DataFrame
        Profile of storage system operation (1 year)
    year: int
        Number of years to simulate. Default 1 year
        
    Output
    ------
    loss_array: numpy-array
        Array with the capacity loss of each timestep in p.u. 
    """
    n = len(df) 
    dt = 15*60 #15min in seconds
    
    
        
        
    soc_array = df["soe1"].to_numpy() #Convert column soc of the DataFrame to a NumPy array
   
    
    temp_array = df["temp"].to_numpy()


#constants
    C_QLOSS = 2.8575
    D_QLOSS = 0.60225
    k_ref = 1.2571e-5
    Eact=17126 #J/mol
    R=8.314 #J/mol*K
    
    #The variables cum_loss and loss_array are initialized. 
    cum_loss   = 0  # cumulative capacity loss in p.u. and is set to 0 initially-----------------------------------------------------------------insert initial loss here
    loss_array = np.zeros(len(df) * years) # capacity loss of each timestep in p.u.( per unit)
    k_temp_array = np.zeros(n * years)
    
    #Return a new array of given shape and type, filled with zeros. (ith a length equal to the number of timesteps multiplied by the number of years.)

    #The code then enters a nested loop, where the outer loop iterates over the range of years specified, and the inner loop iterates over each element (i, soc) in the soc_array.
    for year in range(years):
        for i, (soc, temp) in enumerate(zip(soc_array, temp_array)):
            # calculate stress factor dependent coefficients
            
            k_temp = k_ref*math.exp(-(Eact/R)*((1/temp)-(1/298.15)))
            k_soc = C_QLOSS * (soc - 0.5) ** 3 + D_QLOSS # SOC dependent coefficient
            k = k_temp * k_soc

            # calculate capacity loss per step, based on virtual elapsed time and past total degradation.
            virtual_time = (cum_loss / k)**2 # virtual time in s
            loss = k_soc * k_temp * math.sqrt(virtual_time + dt) # total losses Q
            loss -=  cum_loss # time step losses # subtraction assignment. Subtracts a value from the variable and assigns the result to that variable.
            loss_array[i + n*year] = loss #The calculated loss value is stored in the loss_array at the corresponding index (i + n * year), taking into account the current year.
            k_temp_array[i + n*year] = k_temp
            
            # update variables
            cum_loss += loss  # increase cumulative losses
            

    return loss_array,k_temp_array

In [224]:
def lfp_cyclic(df, years=1):
    """Cyclic-degradation model of LFP cell.
    
    Parameters
    ----------
    df: pandas-DataFrame
        Profile of storage system operation (1 year)
    year: int
        Number of years to simulate. Default 1 year
        
    Output
    ------
    loss_array: numpy-array
        Array with the capacity loss of each timestep in p.u. 
    """
    n =len(df)
    
    # constants
    A_QLOSS = 0.0630
    B_QLOSS = 0.0971
    C_QLOSS = 4.0253
    D_QLOSS = 1.0923

    cum_loss = 0 # cumulative losses
    loss_array = np.zeros(n * years) # capacity loss of each timestep in p.u.
    
    idx_soc = []
        
    #The code then enters a nested loop, where the outer loop iterates over the range of years specified, and the inner loop iterates over each index i and value idx in the DataFrame df.    
    for year in range(years):
        for i, idx in enumerate(df.index):
            doc = df.loc[idx, "doc1"] # depth of cycle in p.u.
            if doc != 0.0:
                # stress factors
                crate = df.loc[idx_soc, "erate1"].mean() # average c-rate of cycle 1/h # 
                delta_fec = doc / 2 #

                # calculate stress factor dependent coefficients
                k_c_rate = A_QLOSS * crate + B_QLOSS
                k_doc = C_QLOSS * (doc - 0.6)**3 + D_QLOSS

                # calculate capacity loss per step, based on virtual FEC and past total degradation.
                virtual_fec = (cum_loss * 100 / (k_c_rate * k_doc))**2
                loss = k_c_rate * k_doc * math.sqrt(virtual_fec + delta_fec) / 100  # total cyc. qloss in p.u.
                loss -= cum_loss  # relative qloss in pu in current timestep
                loss_array[i + n*year] = loss
                
                
                # update variables
                cum_loss += loss
                idx_soc = []    # reset index count
            else:
                idx_soc.append(idx)

    return loss_array

In [225]:
def calc_degradation_strategy(profiles, years=1):
    profile_loss = dict()
    
    # Ensure the directory exists
    loss_folder = os.path.abspath("./data/strategy_losses")
    os.makedirs(loss_folder, exist_ok=True)

    output_folder = os.path.abspath("./data/strategy_temperature_module4")
    
    # Get list of all relevant files in the output folder
    strategy_list = [f for f in os.listdir(output_folder) if f.endswith('.csv')]

    for name, df in profiles.items():
        # Determine the keyword to search for in the strategy list
        strategy_keyword = name.lower()  # Convert to lower case to handle case insensitivity

        for strategy in strategy_list:
            # Check if the strategy keyword is part of the file name
            if strategy_keyword in strategy.lower():  # Convert to lower case for case-insensitive matching
                # Load the matching file
                temp_df = pd.read_csv(os.path.join(output_folder, strategy), header=None)
                temp_df.columns = ["temp"]
                
                # Ensure the length of the module data matches the profile data
                # if len(temp_df) != len(df):
                #     raise ValueError(f"Length of profile data and temp data do not match for {name} and strategy {strategy}.")
                
                # Combine the profile and module data for degradation calculation
                df_combined = pd.concat([df, temp_df], axis=1)
                
                n = len(df_combined)
                idx = pd.date_range(start="2014", periods=n*years, freq="15Min")
                loss = pd.DataFrame(index=idx) # New DataFrame named loss

                # Calculate lfp losses using combined data
                lfp_cal_loss, k_temp = lfp_calendar(df_combined, years=years)
                loss["lfp_calendar1"] = lfp_cal_loss.astype('float64')

                loss["lfp_cyclic1"] = lfp_cyclic(df_combined, years=years).astype('float64')
                loss["lfp1"] = loss["lfp_calendar1"] + loss["lfp_cyclic1"]

                loss["k_temp"]=k_temp

                profile_loss[(name, strategy)] = loss

                # Create the file name using both profile name and strategy
                loss_name = f'loss_{name}_{os.path.splitext(strategy)[0]}.csv'
                file_path = os.path.join(loss_folder, loss_name)

                # Save the subset to a CSV file
                profile_loss[(name, strategy)].to_csv(file_path, index=True, header=True)
                print(f'Saved {loss_name} to {file_path}')
                break  # Stop after finding the first match
        else:
            print(f'No matching file found for profile {name} in {output_folder}.')

    return profile_loss

In [226]:
string_loss = calc_degradation_strategy(string_profiles,  years=1)

Saved loss_MPC_00. M4_temp_allstrategies_Temperature_M4_MPC.csv to c:\Users\granermm\OneDrive - Hochschule für angewandte Wissenschaften Kempten\General\03 Vivek\IRES\JoES IRES Data\data\strategy_losses\loss_MPC_00. M4_temp_allstrategies_Temperature_M4_MPC.csv
Saved loss_Conseq_00. M4_temp_allstrategies_Temperature_M4_Conseq.csv to c:\Users\granermm\OneDrive - Hochschule für angewandte Wissenschaften Kempten\General\03 Vivek\IRES\JoES IRES Data\data\strategy_losses\loss_Conseq_00. M4_temp_allstrategies_Temperature_M4_Conseq.csv
Saved loss_Equal_00. M4_temp_allstrategies_Temperature_M4_Equal.csv to c:\Users\granermm\OneDrive - Hochschule für angewandte Wissenschaften Kempten\General\03 Vivek\IRES\JoES IRES Data\data\strategy_losses\loss_Equal_00. M4_temp_allstrategies_Temperature_M4_Equal.csv
Saved loss_rulebased_00. M4_temp_allstrategies_Temperature_M4_RuleBased.csv to c:\Users\granermm\OneDrive - Hochschule für angewandte Wissenschaften Kempten\General\03 Vivek\IRES\JoES IRES Data\dat

In [227]:
def plot_soh(profiles):
    n = len(profiles)
    
    subplot_titles = tuple(f"{name[1]}" for name in profiles.keys())
    fig = make_subplots(cols=n, subplot_titles=subplot_titles)
    fig.update_layout(template=template)

    for i, (name, df) in enumerate(profiles.items()):
        print(f"Verarbeite Profil: {name}")
        soh = 1 - np.cumsum(df["lfp1"].values.astype('float64'))
         # Debug-Ausgabe: Daten überprüfen
        print(f"Profil: {name}")
        print(df)
        print(soh)
        
        fig.add_trace(go.Scatter(x=df.index, y=soh, name=f"{name} - lfp1"), row=1, col=1+i)
        
        fig.update_yaxes(title="SOH1 in p.u.", range=[0.9994, 1])

    return fig

In [228]:
def plot_k_temp(profiles):
    n = len(profiles)
    
    subplot_titles = tuple(f"{name[1]}" for name in profiles.keys())
    fig = make_subplots(cols=n, subplot_titles=subplot_titles)
    fig.update_layout(template=template)

    for i, (name, df) in enumerate(profiles.items()):
        print(f"Verarbeite Profil: {name}")
        k_temp = np.cumsum(df["k_temp"].values.astype('float64'))
         # Debug-Ausgabe: Daten überprüfen
        print(f"Profil: {name}")
        print(df)
       
        
        fig.add_trace(go.Scatter(x=df.index, y=k_temp, name=f"{name} - lfp1"), row=1, col=1+i)
        
        fig.update_yaxes(title="k_temp stress factor")

    return fig

In [229]:
def plot_loss_rate(profiles):
    n = len(profiles)
    
    subplot_titles = tuple(f"{name[1]}" for name in profiles.keys())
    fig = make_subplots(cols=n, subplot_titles=subplot_titles)
    fig.update_layout(template=template)

    for i, (name, df) in enumerate(profiles.items()):
        print(f"Verarbeite Profil: {name}")
        loss = np.cumsum(df["lfp1"].values.astype('float64'))
         # Debug-Ausgabe: Daten überprüfen
        print(f"Profil: {name}")
        print(df)
        print(loss)
        
        fig.add_trace(go.Scatter(x=df.index, y=loss, name=f"{name} - lfp1"), row=1, col=1+i)
        
        fig.update_yaxes(title="loss in p.u.")

    return fig

In [230]:
# task
def plot_qloss(profiles):
    n = len(profiles)
    #fig = make_subplots(cols=n, subplot_titles=tuple(profiles))
    subplot_titles = tuple(str(title) for title in profiles)
    fig = make_subplots(cols=n, subplot_titles=subplot_titles)
    fig.update_layout(barmode='group', template=template) #changed to group from stack

    # todo color
    for i, (name, df) in enumerate(profiles.items()):
        
        fig.add_trace(go.Bar(name=f"{name} - calendaric", x=["lfp1"], y=df[[ "lfp_calendar1"]].sum()), row=1, col=1+i)
        fig.add_trace(go.Bar(name=f"{name} - cyclic",     x=["lfp1"], y=df[[ "lfp_cyclic1"]].sum()), row=1, col=1+i)
        fig.update_yaxes(title="Q-loss in p.u.")
        

    return fig

In [231]:
string_loss

{('MPC',
  '00. M4_temp_allstrategies_Temperature_M4_MPC.csv'):                      lfp_calendar1  lfp_cyclic1          lfp1        k_temp
 2014-01-01 00:00:00   2.593752e-42     0.000000  2.593752e-42  1.435590e-43
 2014-01-01 00:15:00   1.074664e-42     0.000000  1.074664e-42  1.435822e-43
 2014-01-01 00:30:00   8.245901e-43     0.000002  2.336194e-06  1.435833e-43
 2014-01-01 00:45:00   6.951675e-43     0.000000  6.951675e-43  1.435856e-43
 2014-01-01 01:00:00   7.748175e-43     0.000000  7.748175e-43  1.626903e-43
 ...                            ...          ...           ...           ...
 2014-01-25 22:30:00            NaN          NaN           NaN           NaN
 2014-01-25 22:45:00            NaN          NaN           NaN           NaN
 2014-01-25 23:00:00            NaN          NaN           NaN           NaN
 2014-01-25 23:15:00            NaN          NaN           NaN           NaN
 2014-01-25 23:30:00            NaN          NaN           NaN           NaN
 
 [2399 rows

In [232]:
plot_soh(string_loss)

Verarbeite Profil: ('MPC', '00. M4_temp_allstrategies_Temperature_M4_MPC.csv')
Profil: ('MPC', '00. M4_temp_allstrategies_Temperature_M4_MPC.csv')
                     lfp_calendar1  lfp_cyclic1          lfp1        k_temp
2014-01-01 00:00:00   2.593752e-42     0.000000  2.593752e-42  1.435590e-43
2014-01-01 00:15:00   1.074664e-42     0.000000  1.074664e-42  1.435822e-43
2014-01-01 00:30:00   8.245901e-43     0.000002  2.336194e-06  1.435833e-43
2014-01-01 00:45:00   6.951675e-43     0.000000  6.951675e-43  1.435856e-43
2014-01-01 01:00:00   7.748175e-43     0.000000  7.748175e-43  1.626903e-43
...                            ...          ...           ...           ...
2014-01-25 22:30:00            NaN          NaN           NaN           NaN
2014-01-25 22:45:00            NaN          NaN           NaN           NaN
2014-01-25 23:00:00            NaN          NaN           NaN           NaN
2014-01-25 23:15:00            NaN          NaN           NaN           NaN
2014-01-25 23:30:

Profil: ('Equal', '00. M4_temp_allstrategies_Temperature_M4_Equal.csv')
                     lfp_calendar1   lfp_cyclic1          lfp1        k_temp
2014-01-01 00:00:00   2.593812e-42  0.000000e+00  2.593812e-42  1.435623e-43
2014-01-01 00:15:00   1.074844e-42  1.001389e-05  1.001389e-05  1.435977e-43
2014-01-01 00:30:00   8.282709e-43  0.000000e+00  8.282709e-43  1.439401e-43
2014-01-01 00:45:00   7.058960e-43  0.000000e+00  7.058960e-43  1.448280e-43
2014-01-01 01:00:00   6.881316e-43  0.000000e+00  6.881316e-43  1.529246e-43
...                            ...           ...           ...           ...
2014-01-25 22:30:00   4.625053e-35  0.000000e+00  4.625053e-35  5.440098e-35
2014-01-25 22:45:00   4.513839e-35  0.000000e+00  4.513839e-35  5.407896e-35
2014-01-25 23:00:00   4.429378e-35  0.000000e+00  4.429378e-35  5.363811e-35
2014-01-25 23:15:00   4.315337e-35  3.397545e-09  3.397545e-09  5.321392e-35
2014-01-25 23:30:00   4.210046e-35  0.000000e+00  4.210046e-35  5.261443e-35

[23

In [233]:
plot_loss_rate(string_loss)

Verarbeite Profil: ('MPC', '00. M4_temp_allstrategies_Temperature_M4_MPC.csv')
Profil: ('MPC', '00. M4_temp_allstrategies_Temperature_M4_MPC.csv')
                     lfp_calendar1  lfp_cyclic1          lfp1        k_temp
2014-01-01 00:00:00   2.593752e-42     0.000000  2.593752e-42  1.435590e-43
2014-01-01 00:15:00   1.074664e-42     0.000000  1.074664e-42  1.435822e-43
2014-01-01 00:30:00   8.245901e-43     0.000002  2.336194e-06  1.435833e-43
2014-01-01 00:45:00   6.951675e-43     0.000000  6.951675e-43  1.435856e-43
2014-01-01 01:00:00   7.748175e-43     0.000000  7.748175e-43  1.626903e-43
...                            ...          ...           ...           ...
2014-01-25 22:30:00            NaN          NaN           NaN           NaN
2014-01-25 22:45:00            NaN          NaN           NaN           NaN
2014-01-25 23:00:00            NaN          NaN           NaN           NaN
2014-01-25 23:15:00            NaN          NaN           NaN           NaN
2014-01-25 23:30:

In [234]:
plot_k_temp(string_loss)

Verarbeite Profil: ('MPC', '00. M4_temp_allstrategies_Temperature_M4_MPC.csv')
Profil: ('MPC', '00. M4_temp_allstrategies_Temperature_M4_MPC.csv')
                     lfp_calendar1  lfp_cyclic1          lfp1        k_temp
2014-01-01 00:00:00   2.593752e-42     0.000000  2.593752e-42  1.435590e-43
2014-01-01 00:15:00   1.074664e-42     0.000000  1.074664e-42  1.435822e-43
2014-01-01 00:30:00   8.245901e-43     0.000002  2.336194e-06  1.435833e-43
2014-01-01 00:45:00   6.951675e-43     0.000000  6.951675e-43  1.435856e-43
2014-01-01 01:00:00   7.748175e-43     0.000000  7.748175e-43  1.626903e-43
...                            ...          ...           ...           ...
2014-01-25 22:30:00            NaN          NaN           NaN           NaN
2014-01-25 22:45:00            NaN          NaN           NaN           NaN
2014-01-25 23:00:00            NaN          NaN           NaN           NaN
2014-01-25 23:15:00            NaN          NaN           NaN           NaN
2014-01-25 23:30: